In [1]:
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
import re
import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
#data ingestion
df = pd.read_csv("https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv")

In [3]:
df = df[df['sentiment'].isin(["happiness", "sadness"])]

In [4]:
#data preprocessing
def lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

def remove_stop_words(text):
    stop_words = set(stopwords.words("english"))
    return " ".join([word for word in text.split() if word not in stop_words])

def removing_numbers(text):
    return ''.join([char for char in text if not char.isdigit()])

def lower_case(text):
    return " ".join([word.lower() for word in text.split()])

def removing_punctuations(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    return re.sub('\s+', ' ', text).strip()

def removing_urls(text):
    return re.sub(r'https?://\S+|www\.\S+', '', text)

def remove_small_sentences(df):
    df['content'] = df['content'].apply(lambda x: np.nan if len(str(x).split()) < 3 else x)
    return df

def normalize_text(df):
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        df = remove_small_sentences(df)
        return df.dropna(subset=['content'])
    except Exception as e:
        print(e)

In [5]:
df = normalize_text(df)

In [6]:
df['sentiment'].replace({
    'sadness':0,
    'happiness':1
}, inplace=True)

In [7]:
#apply bow
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
#log experiment on dagshub mlflow
import dagshub

dagshub.init(repo_owner='iamprashantjain', repo_name='mini_project', mlflow=True)
mlflow.set_tracking_uri("https://dagshub.com/iamprashantjain/mini_project.mlflow")
mlflow.set_experiment("logistic regression baseline")

Accessing as iamprashantjain

Initialized MLflow to track repo "iamprashantjain/mini_project"

Repository iamprashantjain/mini_project initialized!

<Experiment: artifact_location='mlflow-artifacts:/5999e58740034acf8100bfcf55c7c783', creation_time=1748369261099, experiment_id='0', last_update_time=1748369261099, lifecycle_stage='active', name='logistic regression baseline', tags={}>

In [10]:
#now run experiment
with mlflow.start_run():
    #log preprocessing params
    mlflow.log_param("vectorizer","bow")
    mlflow.log_param("num_features","1000")
    mlflow.log_param("test_size",0.2)
    
    #model training
    model = LogisticRegression(solver='saga')
    model.fit(X_train, y_train)
    
    #log model params
    mlflow.log_param("model", "LogisticRegression")
    
    #model evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    
    #log evaluation metric
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    
    
    #log model
    mlflow.sklearn.log_model(model, "model")
    
    
    #log notebook
    import os
    notebook_path = "exp1.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)
    
    
    print("accuracy", accuracy)
    print("precision", precision)
    print("recall", recall)
    print("f1", f1)    

c:\users\iampr\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


2025/05/28 17:12:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


accuracy 0.7666666666666667
precision 0.7561214495592556
recall 0.783756345177665
f1 0.769690927218345


2025/05/28 17:15:15 INFO mlflow.tracking._tracking_service.client: 🏃 View run useful-whale-721 at: https://dagshub.com/iamprashantjain/mini_project.mlflow/#/experiments/0/runs/ff0a3f7a60d145d79bd50ce20ea2642e.


2025/05/28 17:15:15 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/iamprashantjain/mini_project.mlflow/#/experiments/0.
